<a href="https://colab.research.google.com/github/427Stanley427/Python/blob/main/btckeystxt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bitcoin

In [ ]:
pip install requests

In [ ]:
import requests
import bitcoin

def generate_bitcoin_keypair_with_prefix(prefix):
    counter = 0  # Compteur pour suivre le nombre de clés générées
    file_counter = 1  # Compteur pour les noms de fichiers
    filename = f"adresse{file_counter}.txt"
    file = open(filename, "w")

    while True:
        private_key = bitcoin.random_key()
        public_key_uncompressed = bitcoin.privtopub(private_key)
        address_uncompressed = bitcoin.pubtoaddr(public_key_uncompressed)

        wif_private_key = bitcoin.encode_privkey(bitcoin.decode_privkey(private_key, 'hex'), 'wif')

        # Sauvegarde des détails dans le fichier
        file.write(f"--- Tentative #{counter + 1} ---\n")
        file.write(f"Clé privée WIF: {wif_private_key}\n")
        file.write(f"Adresse non compressée: {address_uncompressed}\n")

        public_key_compressed = bitcoin.compress(bitcoin.decode_pubkey(public_key_uncompressed, 'hex'))
        address_compressed = bitcoin.pubtoaddr(public_key_compressed)
        file.write(f"Adresse compressée: {address_compressed}\n\n")

        # Affichage des détails
        print(f"--- Tentative #{counter + 1} ---")
        print(f"Clé privée WIF: {wif_private_key}")
        print(f"Adresse non compressée: {address_uncompressed}")
        print(f"Adresse compressée: {address_compressed}")

        if address_uncompressed.startswith(prefix):
            file.close()
            return {
                "private_key_hex": private_key,
                "wif_private_key": wif_private_key,
                "public_key_uncompressed": public_key_uncompressed,
                "public_key_compressed": public_key_compressed,
                "address_uncompressed": address_uncompressed,
                "address_compressed": address_compressed
            }

        counter += 1

        # Si nous avons atteint 50000 adresses, nous créons un nouveau fichier
        if counter % 50000 == 0:
            file_counter += 1
            file.close()
            filename = f"adresse{file_counter}.txt"
            file = open(filename, "w")

def get_balance(address):
    url = f"https://api.blockchair.com/bitcoin/dashboards/address/{address}"
    response = requests.get(url)
    data = response.json().get("data", {})

    if not data:
        return 0

    address_data = data.get(address, {})
    if not address_data:
        return 0

    balance_satoshis = address_data.get("address", {}).get("balance", 0)
    return balance_satoshis / 100_000_000  # Convertir en BTC

kp = generate_bitcoin_keypair_with_prefix("1X")
balance_uncompressed = get_balance(kp["address_uncompressed"])
balance_compressed = get_balance(kp["address_compressed"])

print("Clé privée hex :", kp["private_key_hex"])
print("Clé privée WIF :", kp["wif_private_key"])
print("Clé publique non compressée :", kp["public_key_uncompressed"])
print("Clé publique compressée :", kp["public_key_compressed"])
print("Adresse non compressée :", kp["address_uncompressed"], "| Balance:", balance_uncompressed, "BTC")
print("Adresse compressée :", kp["address_compressed"], "| Balance:", balance_compressed, "BTC")
